In [ ]:
import tensorflow as tf
print('tf version : ' + tf.__version__)

tf version : 2.3.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Unzip CovidX-v4 dataset

# unzip Dr Arganda-Carreras's file:
# !unzip -q '/content/drive/My Drive/Colab Notebooks/Internship/COVID19_DATA/Covid-X-v4.zip'

# unzip mine (a copy of Dr Arganda-Carreras's file)
!unzip -q '/content/drive/My Drive/Colab Notebooks/Internship/COVIDX_models/Covid-X-v4.zip'

print('Done!')

replace data/test_split.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: Done!


# Nouvelle section

In [ ]:
from keras.models import Sequential, save_model, load_model
from keras import regularizers, optimizers, Model

from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.callbacks import *

from sklearn.model_selection import StratifiedKFold, KFold
import logging

logging.getLogger('tensorflow').setLevel(logging.ERROR)


class CosineAnnealer:
    
    def __init__(self, start, end, steps):
        self.start = start
        self.end = end
        self.steps = steps
        self.n = 0
        
    def step(self):
        self.n += 1
        cos = np.cos(np.pi * (self.n / self.steps)) + 1
        return self.end + (self.start - self.end) / 2. * cos


class OneCycleScheduler(Callback):
    """`Callback` that schedules the learning rate on a 1cycle policy as per Leslie Smith's paper(https://arxiv.org/pdf/1803.09820.pdf).
    If the model supports a momentum parameter, it will also be adapted by the schedule.
    The implementation adopts additional improvements as per the fastai library: https://docs.fast.ai/callbacks.one_cycle.html, where
    only two phases are used and the adaptation is done using cosine annealing.
    In phase 1 the LR increases from `lr_max / div_factor` to `lr_max` and momentum decreases from `mom_max` to `mom_min`.
    In the second phase the LR decreases from `lr_max` to `lr_max / (div_factor * 1e4)` and momemtum from `mom_max` to `mom_min`.
    By default the phases are not of equal length, with the phase 1 percentage controlled by the parameter `phase_1_pct`.
    """

    def __init__(self, lr_max, steps, mom_min=0.85, mom_max=0.95, phase_1_pct=0.3, div_factor=25.):
        super(OneCycleScheduler, self).__init__()
        lr_min = lr_max / div_factor
        final_lr = lr_max / (div_factor * 1e4)
        phase_1_steps = steps * phase_1_pct
        phase_2_steps = steps - phase_1_steps
        
        self.phase_1_steps = phase_1_steps
        self.phase_2_steps = phase_2_steps
        self.phase = 0
        self.step = 0
        
        self.phases = [[CosineAnnealer(lr_min, lr_max, phase_1_steps), CosineAnnealer(mom_max, mom_min, phase_1_steps)], 
                 [CosineAnnealer(lr_max, final_lr, phase_2_steps), CosineAnnealer(mom_min, mom_max, phase_2_steps)]]
        
        self.lrs = []
        self.moms = []

    def on_train_begin(self, logs=None):
        self.phase = 0
        self.step = 0

        self.set_lr(self.lr_schedule().start)
        self.set_momentum(self.mom_schedule().start)
        
    def on_train_batch_begin(self, batch, logs=None):
        self.lrs.append(self.get_lr())
        self.moms.append(self.get_momentum())

    def on_train_batch_end(self, batch, logs=None):
        self.step += 1
        if self.step >= self.phase_1_steps:
            self.phase = 1
            
        self.set_lr(self.lr_schedule().step())
        self.set_momentum(self.mom_schedule().step())
        
    def get_lr(self):
        try:
            return tf.keras.backend.get_value(self.model.optimizer.lr)
        except AttributeError:
            return None
        
    def get_momentum(self):
        try:
            return tf.keras.backend.get_value(self.model.optimizer.momentum)
        except AttributeError:
            return None
        
    def set_lr(self, lr):
        try:
            tf.keras.backend.set_value(self.model.optimizer.lr, lr)
        except AttributeError:
            pass # ignore
        
    def set_momentum(self, mom):
        try:
            tf.keras.backend.set_value(self.model.optimizer.momentum, mom)
        except AttributeError:
            pass # ignore

    def lr_schedule(self):
        return self.phases[self.phase][0]
    
    def mom_schedule(self):
        return self.phases[self.phase][1]
    
    def plot(self):
        ax = plt.subplot(1, 2, 1)
        ax.plot(self.lrs)
        ax.set_title('Learning Rate')
        ax = plt.subplot(1, 2, 2)
        ax.plot(self.moms)
        ax.set_title('Momentum')


class LRFinder(Callback):
    """`Callback` that exponentially adjusts the learning rate after each training batch between `start_lr` and
    `end_lr` for a maximum number of batches: `max_step`. The loss and learning rate are recorded at each step allowing
    visually finding a good learning rate as per https://sgugger.github.io/how-do-you-find-a-good-learning-rate.html via
    the `plot` method.
    """

    def __init__(self, start_lr: float = 1e-7, end_lr: float = 10, max_steps: int = 1000, smoothing=0.9):
        super(LRFinder, self).__init__()
        self.start_lr, self.end_lr = start_lr, end_lr
        self.max_steps = max_steps
        self.smoothing = smoothing
        self.step, self.best_loss, self.avg_loss, self.lr = 0, 0, 0, 0
        self.lrs, self.losses = [], []

    def on_train_begin(self, logs=None):
        self.step, self.best_loss, self.avg_loss, self.lr = 0, 0, 0, 0
        self.lrs, self.losses = [], []

    def on_train_batch_begin(self, batch, logs=None):
        self.lr = self.exp_annealing(self.step)
        tf.keras.backend.set_value(self.model.optimizer.lr, self.lr)

    def on_train_batch_end(self, batch, logs=None):
        logs = logs or {}
        loss = logs.get('loss')
        step = self.step
        if loss:
            self.avg_loss = self.smoothing * self.avg_loss + (1 - self.smoothing) * loss
            smooth_loss = self.avg_loss / (1 - self.smoothing ** (self.step + 1))
            self.losses.append(smooth_loss)
            self.lrs.append(self.lr)

            if step == 0 or loss < self.best_loss:
                self.best_loss = loss

            if smooth_loss > 4 * self.best_loss or tf.math.is_nan(smooth_loss):
                self.model.stop_training = True

        if step == self.max_steps:
            self.model.stop_training = True

        self.step += 1

    def exp_annealing(self, step):
        return self.start_lr * (self.end_lr / self.start_lr) ** (step * 1. / self.max_steps)

    def plot(self):
        fig, ax = plt.subplots(1, 1)
        ax.set_ylabel('Loss')
        ax.set_xlabel('Learning Rate (log scale)')
        ax.set_xscale('log')
        ax.xaxis.set_major_formatter(plt.FormatStrFormatter('%.0e'))
        ax.plot(self.lrs, self.losses)


In [ ]:
# copied from https://github.com/kobiso/CBAM-keras/blob/master/models/attention_module.py
# used for Breastnet model : https://github.com/Goodsea/BreastNet/blob/master/40X/BreastNet_40X.ipynb adapted for keras

from keras.layers import *
from keras.optimizers import *
from keras.models import load_model, Model

from keras import backend as K
SHAPE = (224, 224, 3)
from keras.layers import Input

def cbam_block(cbam_feature, ratio=8):
    """Contains the implementation of Convolutional Block Attention Module(CBAM) block.
    As described in https://arxiv.org/abs/1807.06521.
    """
    
    cbam_feature = channel_attention(cbam_feature, ratio)
    cbam_feature = spatial_attention(cbam_feature)
    return cbam_feature

def channel_attention(input_feature, ratio=8):
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    channel = input_feature._shape[channel_axis] #change _keras_shape into _shape
    
    shared_layer_one = Dense(channel//ratio,
                             activation='relu',
                             kernel_initializer='he_normal',
                             use_bias=True,
                             bias_initializer='zeros')
    shared_layer_two = Dense(channel,
                             kernel_initializer='he_normal',
                             use_bias=True,
                             bias_initializer='zeros')
    
    avg_pool = GlobalAveragePooling2D()(input_feature)    
    avg_pool = Reshape((1,1,channel))(avg_pool)
    assert avg_pool._shape[1:] == (1,1,channel) #_keras
    avg_pool = shared_layer_one(avg_pool)
    assert avg_pool._shape[1:] == (1,1,channel//ratio) #_keras
    avg_pool = shared_layer_two(avg_pool)
    assert avg_pool._shape[1:] == (1,1,channel) #_keras
    
    max_pool = GlobalMaxPooling2D()(input_feature)
    max_pool = Reshape((1,1,channel))(max_pool)
    assert max_pool._shape[1:] == (1,1,channel) #_keras
    max_pool = shared_layer_one(max_pool)
    assert max_pool._shape[1:] == (1,1,channel//ratio) #_keras
    max_pool = shared_layer_two(max_pool)
    assert max_pool._shape[1:] == (1,1,channel) #_keras
    
    cbam_feature = Add()([avg_pool,max_pool])
    cbam_feature = Activation('sigmoid')(cbam_feature)

    if K.image_data_format() == "channels_first":
        cbam_feature = Permute((3, 1, 2))(cbam_feature)
    
    return multiply([input_feature, cbam_feature])

def spatial_attention(input_feature):
    kernel_size = 7
    
    if K.image_data_format() == "channels_first":
        channel = input_feature._shape[1] #_keras
        cbam_feature = Permute((2,3,1))(input_feature)
    else:
        channel = input_feature._shape[-1] #_keras_
        cbam_feature = input_feature
    
    avg_pool = Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(cbam_feature)
    assert avg_pool._shape[-1] == 1 #_keras
    max_pool = Lambda(lambda x: K.max(x, axis=3, keepdims=True))(cbam_feature)
    assert max_pool._shape[-1] == 1 #_keras
    concat = Concatenate(axis=3)([avg_pool, max_pool])
    assert concat._shape[-1] == 2 #_keras
    cbam_feature = Conv2D(filters = 1,
                    kernel_size=kernel_size,
                    strides=1,
                    padding='same',
                    activation='sigmoid',
                    kernel_initializer='he_normal',
                    use_bias=False)(concat)	
    assert cbam_feature._shape[-1] == 1 #_keras
    
    if K.image_data_format() == "channels_first":
        cbam_feature = Permute((3, 1, 2))(cbam_feature)
        
    return multiply([input_feature, cbam_feature])

In [ ]:

# credits: https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras

def recall(y_true, y_pred):
    """
    Recall metric.
    
    Only computes a batch-wise average of recall.
    
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    """Precision metric.
    
    Only computes a batch-wise average of precision.
    
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):
    precisionx = precision(y_true, y_pred)
    recallx = recall(y_true, y_pred)
    return 2*((precisionx*recallx)/(precisionx+recallx+K.epsilon()))

In [ ]:
# copied from https://gist.github.com/mjdietzx/5319e42637ed7ef095d430cb5c5e8c64
def residual_block(y, nb_channels, _strides=(1, 1), _project_shortcut=False):
    shortcut = y

    # down-sampling is performed with a stride of 2
    y = Conv2D(nb_channels, kernel_size=(3, 3), strides=_strides, padding='same')(y)
    y = BatchNormalization()(y)
    y = LeakyReLU()(y)

    y = Conv2D(nb_channels, kernel_size=(3, 3), strides=(1, 1), padding='same')(y)
    y = BatchNormalization()(y)

    # identity shortcuts used directly when the input and output are of the same dimensions
    if _project_shortcut or _strides != (1, 1):
        # when the dimensions increase projection shortcut is used to match dimensions (done by 1×1 convolutions)
        # when the shortcuts go across feature maps of two sizes, they are performed with a stride of 2
        shortcut = Conv2D(nb_channels, kernel_size=(1, 1), strides=_strides, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)

    y = add([shortcut, y])
    y = LeakyReLU()(y)

    return y


In [ ]:
# copied from https://github.com/Goodsea/BreastNet/blob/master/40X/BreastNet_40X.ipynb, "create_model()"

# We adapt this model by adding some dropout layers to counter overfitting
def Breastnet_modified_model():
    
    dropRate = 0.3
    
    init = Input(SHAPE)
    x = Conv2D(32, (3, 3), activation=None, padding='same')(init) 
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(dropRate)(x) # adding dropout
    x = Conv2D(32, (3, 3), activation=None, padding='same')(x) 
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(dropRate)(x) # adding dropout
    x1 = MaxPooling2D((2,2))(x)
    
    x = Conv2D(64, (3, 3), activation=None, padding='same')(x1)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(dropRate)(x) # adding dropout
    x = cbam_block(x)
    x = residual_block(x, 64)
    x2 = MaxPooling2D((2,2))(x)
    
    x = Conv2D(128, (3, 3), activation=None, padding='same')(x2)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(dropRate)(x) # adding dropout
    x = cbam_block(x)
    x = residual_block(x, 128)
    x3 = MaxPooling2D((2,2))(x)
    
    ginp1 = UpSampling2D(size=(2, 2), interpolation='bilinear')(x1)
    ginp2 = UpSampling2D(size=(4, 4), interpolation='bilinear')(x2)
    ginp3 = UpSampling2D(size=(8, 8), interpolation='bilinear')(x3)
    
    hypercolumn = Concatenate()([ginp1, ginp2, ginp3]) 
    gap = GlobalAveragePooling2D()(hypercolumn)

    x = Dense(256, activation=None)(gap)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(dropRate)(x) # this one was already in the original model
    
    x = Dense(256, activation=None)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    y = Dense(3, activation='softmax')(x) # modified it for 3 classes 
   
    model = Model(init, y)
    return model

In [ ]:
def create_model():
  # CLR parameters
    lr = 5e-3
    epochs = 5

    STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
    STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

    steps = STEP_SIZE_TRAIN * epochs

    lr_schedule = OneCycleScheduler(lr, steps)

    base_model = Breastnet_modified_model()

    optimizer = tf.keras.optimizers.Adam(lr=lr)

    # optimizer = tf.keras.optimizers.Adam(amsgrad=True)
    base_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics= [precision, recall, f1, 'accuracy']) #, weighted_metrics = class_weights)

    # base_model.summary()
    return base_model


# data

In [ ]:
import pathlib
path = pathlib.Path( '/content/data' )

# Parameters

# Model path
mod_path = '/content/drive/My Drive/Colab Notebooks/Internship/proto1'

k_folds = 4
# batch size
bs = 64

import pandas as pd
import numpy as np
import keras
from keras_preprocessing.image import ImageDataGenerator
import io


from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedShuffleSplit

# Paths
path_train = pathlib.Path('/content/data/train/')
path_test = pathlib.Path('/content/data/test/')


# initialize the data and labels
data = []
labels = []

traindf= pd.read_csv(path/"train_split.txt", dtype=str, sep=' ', header=None, names=['Image','Name','Class','Origin']) #.drop('Origin',axis=1).drop('Image',axis=1)
testdf=pd.read_csv(path/"test_split.txt", dtype=str, sep=' ', header=None, names=['Image','Name','Class', 'Origin']) #.drop('Origin',axis=1).drop('Image',axis=1)

traindf = traindf[['Name', 'Class']]
testdf = testdf[['Name','Class']]




In [ ]:
# adapted from https://www.kaggle.com/janged/3rd-ml-month-xception-stratifiedkfold-ensemble

from collections import Counter

skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=2019)

# datagen = ImageDataGenerator(rescale=1./255,
#                                 rotation_range=30,
# 	                              zoom_range=0.15,
# 	                              width_shift_range=0.2,
# 	                              height_shift_range=0.2,
# 	                              shear_range=0.15,
# 	                              horizontal_flip=True,
# 	                              fill_mode="nearest")

datagen = ImageDataGenerator(rescale=1./255)


size = 224 # Size of the images


model_breastnet_names = []
j = 1
for (train_index, valid_index) in skf.split(
    traindf['Name'], 
    traindf['Class']):

    traindfb = traindf.iloc[train_index, :].reset_index()
    validdfb = traindf.iloc[valid_index, :].reset_index()

    print("============================================")
    print("====== K Fold Validation step => %d/%d =======" % (j,k_folds))
    print("============================================")
    

    train_generator=datagen.flow_from_dataframe(
      dataframe=traindfb,
      directory=path_train,
      x_col='Name',
      y_col='Class',
      batch_size=bs,
      seed=2019,
      shuffle=True,
      class_mode="categorical",
      target_size=(size,size))

    valid_generator=datagen.flow_from_dataframe(
      dataframe=validdfb,
      directory=path_train,
      x_col='Name',
      y_col='Class',
      batch_size=bs,
      seed=2019,
      shuffle=True,
      class_mode="categorical",
      target_size=(size,size))
    

    # compute class weight:
    labels_count = Counter()

    for word in traindf['Class']:
      labels_count[word] += 1

    total_count = sum(labels_count.values())
    class_weights = {cls: total_count / count for cls, count in labels_count.items()}

    class_weights[train_generator.class_indices['COVID-19']] = class_weights.pop('COVID-19')
    class_weights[train_generator.class_indices['normal']] = class_weights.pop('normal')
    class_weights[train_generator.class_indices['pneumonia']] = class_weights.pop('pneumonia')


    model_name = str(j) + '_breastnet.hdf5'
    model_breastnet_names.append(model_name)
    
    base_model = create_model()

    checkpoint_callback = ModelCheckpoint(
      filepath=model_name,
      monitor='val_loss', 
      mode='min', 
      verbose=1, 
      save_best_only=True, 
      save_weights_only=False)


    History = base_model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    class_weight=class_weights, # We change the weights
                    epochs=epochs,
                    callbacks=[checkpoint_callback, lr_schedule],
                    workers=4,
                    verbose = 2)
        
    j+=1

====== K Fold Validation step => 1/4 =======
Found 10419 validated image filenames belonging to 3 classes.
Found 3473 validated image filenames belonging to 3 classes.
Epoch 1/5

Epoch 00001: val_loss improved from inf to 1.62746, saving model to 1_breastnet.hdf5
162/162 - 130s - loss: 3.7460 - precision: 0.3493 - recall: 0.2351 - f1: 0.2805 - accuracy: 0.3360 - val_loss: 1.6275 - val_precision: 0.0336 - val_recall: 0.0336 - val_f1: 0.0336 - val_accuracy: 0.0336
Epoch 2/5

Epoch 00002: val_loss did not improve from 1.62746
162/162 - 127s - loss: 2.5171 - precision: 0.6277 - recall: 0.4858 - f1: 0.5467 - accuracy: 0.5862 - val_loss: 2.0832 - val_precision: 0.0333 - val_recall: 0.0333 - val_f1: 0.0333 - val_accuracy: 0.0333
Epoch 3/5

Epoch 00003: val_loss did not improve from 1.62746
162/162 - 127s - loss: 2.2808 - precision: 0.7020 - recall: 0.5813 - f1: 0.6355 - accuracy: 0.6539 - val_loss: 2.3511 - val_precision: 0.0342 - val_recall: 0.0341 - val_f1: 0.0342 - val_accuracy: 0.0341
Epo

In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255)

test_generator=test_datagen.flow_from_dataframe(
  dataframe=testdf,
  directory=path_test,
  x_col='Name',
  y_col=None,
  batch_size=bs,
  shuffle=False, #DO NOT SHUFFLE, otherwise you won't be able to calculate monitors afterward 
  class_mode=None,
  target_size=(size,size))


breastnet_prediction = []
for i, name in enumerate(model_breastnet_names):

    model_bn = create_model()

    model_bn.load_weights(name)
    test_generator.reset()
    pred = model_bn.predict_generator(
        generator=test_generator,
        # steps = test_generator.n//test_generator.batch_size,
        verbose=1
    )
    breastnet_prediction.append(pred)

y_pred_breastnet = np.mean(breastnet_prediction, axis=0)

class_guess=np.argmax(y_pred_breastnet, axis=1)

# adapted from COVIDX-Fastai-XResNet18.ipynb
# Convert dataframe test labels to list
gt = testdf['Class'].tolist()

# Convert from label names to class index values (0, 1, 2)
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
labels = ['COVID-19', 'normal', 'pneumonia']
le = preprocessing.LabelEncoder()
targets = le.fit_transform(labels)
test_preds = le.transform( gt )

print("Size of class_guess : " + str(np.size(class_guess)))
print("Size of test_preds : " + str(np.size(np.asarray( test_preds ))))
print("class_guess : " + str(class_guess))
print("test_preds : " + str(test_preds))

# Calculate accuracy
from sklearn.metrics import accuracy_score
acc = accuracy_score( class_guess, np.asarray( test_preds ) )
print( "Accuracy = " + str( acc ) )

# Calculate precision per class
print(labels)

from sklearn.metrics import precision_score
prec = precision_score( class_guess, np.asarray( test_preds ), average=None )
print( "Precision (Positive Predictive Value) per class = " + str( prec ))

# Calculate recall per class
from sklearn.metrics import recall_score
rec = recall_score(class_guess, np.asarray(test_preds), average=None )
print( "Recall (Sensitiviy) per class = " + str( rec ))


print('Confusion Matrix')
print(confusion_matrix(np.asarray( test_preds ), class_guess))


Found 1579 validated image filenames.
25/25 [==============================] - 16s 653ms/step
Size of class_guess : 1579
Size of test_preds : 1579
class_guess : [0 0 0 ... 0 0 0]
test_preds : [2 2 2 ... 2 2 2]
Accuracy = 0.06269791006966434
['COVID-19', 'normal', 'pneumonia']
Precision (Positive Predictive Value) per class = [0.99 0.   0.  ]
Recall (Sensitiviy) per class = [0.06277743 0.         0.        ]
Confusion Matrix
[[ 99   0   1]
 [884   0   1]
 [594   0   0]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
